In [1]:
import sys
sys.path.insert(1, "../input/apxembeddings-utils/")

In [2]:
# Import the required libraries
import os
import torch
import numpy as np
import transformers
from transformers import AutoModelForSequenceClassification, GlueDataTrainingArguments, AutoTokenizer
import torchmetrics
import datasets
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from tqdm import tqdm

from apx import ApproxEmbed, ApxSVD, train_apx, set_random_seed
from glue_score import GLUE_TASKS, make_model, Glue, get_dataloaders, validate, get_metrics

In [3]:
set_random_seed(42)

In [4]:
transformers.utils.logging.set_verbosity_error()
device = "cuda:0" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained('prajjwal1/bert-tiny')

epochs=5
batch_size=32

lr=5e-5
levels=8
channels=8
bits=8
neurons=64
nn_levels=2
apx_epochs=5000
apx_batch_size=2**14
checkpoint_every=100

save_path = 'results/compression_finetuned/'

losses=[('Norm 1',         lambda embed_preds,embeds: torch.mean(torch.pow(torch.abs(embed_preds-embeds),1.))),
        ('Norm 1.25',      lambda embed_preds,embeds: torch.mean(torch.pow(torch.abs(embed_preds-embeds),1.25))),
        ('Norm 1.5',       lambda embed_preds,embeds: torch.mean(torch.pow(torch.abs(embed_preds-embeds),1.5))),
        ('Norm 1.75',      lambda embed_preds,embeds: torch.mean(torch.pow(torch.abs(embed_preds-embeds),1.75))),
        ('Norm 2',         lambda embed_preds,embeds: torch.mean(torch.pow(torch.abs(embed_preds-embeds),2.))),
        ('Norm 1 Cosine',lambda embed_preds,embeds: torch.mean(torch.pow(torch.abs(embed_preds-embeds),1.)) - 50* torch.mean(torch.nn.functional.cosine_similarity(embed_preds,embeds)))
       ]

Downloading:   0%|          | 0.00/285 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

In [5]:
RESULTS={}
for task in list(GLUE_TASKS.keys()):
    
    print(task)
    args=GLUE_TASKS[task]
    model=make_model('prajjwal1/bert-tiny',args)
    
    train_dataloader, val_dataloader = get_dataloaders(args, task, batch_size)
    metrics = get_metrics(args)

    Glue(model, tokenizer, task, args, epochs=epochs, steps_validate=0.2, train_dataloader=train_dataloader, val_dataloader=val_dataloader);

    embeddings = model.bert.embeddings.word_embeddings.weight.to(device)
    
    results={}

    val_metrics = validate(model, tokenizer, val_dataloader, metrics, args)
    results['Original']=[m.item() for m in val_metrics]
    print('Original', val_metrics)

    for name,loss_f in losses:

        # clone the model to not modify the original
        apx_model = make_model('prajjwal1/bert-tiny',args)
        apx_model.load_state_dict(model.state_dict())

        apx = ApproxEmbed(levels = levels, 
                feature_dim = channels,
                num_words = embeddings.shape[0],
                output_dims = embeddings.shape[1],
                feature_std = 0.1,
                feature_bias = 0.0,
                codebook_bitwidth=bits,
                neurons = neurons,
                nn_levels=nn_levels).to('cuda')

        train_apx(apx=apx,
                  embeddings=embeddings, 
                  epochs=apx_epochs, 
                  batch_size=apx_batch_size, 
                  lr=0.01,
                  checkpoint_every=checkpoint_every,
                  save_path=save_path,
                  loss_function=loss_f,
                  name=name);
        
        apx.fix_indices()
        apx_model.bert.embeddings.word_embeddings = apx

        val_metrics = validate(apx_model, tokenizer, val_dataloader, metrics, args)
        results[name]=[m.item() for m in val_metrics]
        print(name, val_metrics)

    # clone the model to not modify the original
    apx_model = make_model('prajjwal1/bert-tiny',args)
    apx_model.load_state_dict(model.state_dict())

    apxSVD = ApxSVD(apx_model.bert.embeddings.word_embeddings.weight, 5)
    apx_model.bert.embeddings.word_embeddings = apxSVD.to('cuda')

    val_metrics = validate(apx_model, tokenizer, val_dataloader, metrics, args)
    results['SVD']=[m.item() for m in val_metrics]
    print('SVD', val_metrics)

    RESULTS[task]=results

cola


Downloading:   0%|          | 0.00/16.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8551 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1043 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1063 [00:00<?, ? examples/s]

...

Generating validation split:   0%|          | 0/71 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/146 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/wnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


Epoch: 5: 100%|██████████| 20/20 [00:00<00:00, 27.22it/s, loss: 0.174]


Original [tensor(0.4625)]
Norm 1 [tensor(0.4585)]
Norm 1.25 [tensor(0.5052)]
Norm 1.5 [tensor(0.4831)]
Norm 1.75 [tensor(0.5089)]
Norm 2 [tensor(0.5000)]
Norm 1 Cosine [tensor(0.4964)]
SVD [tensor(0.4540)]


In [6]:
save_path='results/'
import os
if not os.path.exists(save_path):
    os.makedirs(save_path)

torch.save(RESULTS, save_path+'glue_results_no_retrain.pth')

In [7]:
import torch
import pandas as pd

run_dict=torch.load('results/glue_results_no_retrain.pth')

for k_runs, runs in run_dict.items():
    for k_run, run in runs.items():
        run_dict[k_runs][k_run]=f"{run[0]*100:,.1f}" if len(run)==1 else f"{run[0]*100:,.1f}/{run[1]*100:,.1f}"
df=pd.DataFrame(run_dict)
df

,cola,sst2,mrpc,stsb,qqp,mnli,qnli,rte,wnli
Original,0.0,80.9,52.4/55.4,36.2/35.2,81.6,65.0/63.6,79.8,54.1,46.3
Norm 1,0.0,70.8,40.6/50.0,10.4/9.2,48.3,33.3/33.3,50.1,50.4,45.8
Norm 1.25,0.0,70.0,40.6/50.0,13.1/11.9,53.4,33.3/33.4,50.0,50.0,50.5
Norm 1.5,0.0,70.9,40.6/50.0,14.6/14.3,50.1,34.0/34.2,50.0,50.0,48.3
Norm 1.75,0.0,69.9,40.6/50.0,11.2/10.0,51.7,33.3/33.4,50.4,50.0,50.9
Norm 2,0.0,73.7,40.6/50.0,8.0/7.1,48.6,33.5/33.9,50.9,50.0,50.0
Norm 1 Cosine,0.0,70.8,40.6/50.0,14.3/13.4,27.1,33.3/33.3,68.5,50.0,49.6
SVD,0.0,51.1,40.6/50.0,-0.6/-1.1,39.6,33.2/33.3,52.1,50.4,45.4
